In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

import scipy
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from scipy.optimize import SR1, BFGS
from scipy.optimize import minimize

import confound_mdp
import confound_ope
import confound_env

from core.sepsisSimDiabetes.State import State
from core.sepsisSimDiabetes.Action import Action
from core import generator_confounded_mdp as DGEN
from core import conf_wis as CWIS
from core import loss_minimization as LB
from utils.utils import *

import gurobipy as gp
from gurobipy import GRB

# Comparison with candidate MDP for gridworld

In [ ]:
envs = []
# each row:
#   [mdp , pi_b, pi_e, horizon, gamma, nStates, nActions, term]

horizon = 8
pi_b, P, R, x_dist, u_dist, gamma = confound_env.gridworld_opetools(horizon = horizon, slip = 0.04, confound_weight=0.6)
#R = -1*R
gridworld = confound_mdp.ConfoundMDP(P, R, x_dist, u_dist, gamma)

nStates = P.shape[2]
nActions = P.shape[1]

pi_e = np.zeros((nStates, nActions))
for i in range(nStates):
    pi_e[i] = [0.4, 0.1, 0.4, 0.1]
    
envs.append([gridworld, pi_b, pi_e, horizon, gamma, nStates, nActions, -1])

In [ ]:
def fixed_u_gp_s_rect_s(s, y, pi_e, u_param, Phat, pihat, P_bound, pi_bound, mdp):
    nStates = mdp.n_states
    nActions = mdp.n_actions
    nU = mdp.n_confound

    m = gp.Model("bilinear")
    m.setParam('OutputFlag', 0) 
    
    u_dist = np.array([1-u_param, u_param])
    
    P = m.addMVar((nU,nStates, nActions), lb=0.0, ub=1.0) 
    pi = m.addMVar((nU,nActions), lb=0.0, ub=1.0)
    
    #P.Start = np.swapaxes(P_est[:,:,s,:], 1, 2)
    
    for a in range(nActions):
        prepiupper = pi_bound/pihat[s,a] + (1-pi_bound)
        prepilower = 1/(pi_bound*pihat[s,a]) + (1 - 1/pi_bound)
        m.addConstr(pi[:,a] >= 1/prepiupper)
        m.addConstr(pi[:,a] <= 1/prepilower)
    
    for a in range(nActions):
        for i in range(nStates):
            if Phat[a,s,i] == 0:
                m.addConstr(P[:,i,a] >= 0)
                m.addConstr(P[:,i,a] <= 0)
            else:
                prePupper = P_bound/Phat[a, s, i] + (1-P_bound)
                prePlower = 1/(P_bound*Phat[a, s, i]) + (1 - 1/P_bound)
                m.addConstr(P[:,i,a] >= max(0,1/prePupper))
                m.addConstr(P[:,i,a] <= min(1/prePlower, 1))

    for a in range(nActions):
        for u in range(nU):
            m.addConstr(P[u, :, a] @ np.ones(nStates) == 1)

    for a in range(nActions):
        for i in range(nStates):
            m.addConstr((pi[:,a] @ np.diag(u_dist)) @ P[:,i, a] == pihat[s,a] * Phat[a,s,i])

    for a in range(nActions):
        m.addConstr(pi[:,a] @ u_dist == pihat[s,a])
        
    for u in range(nU):
        m.addConstr(pi[u,:] @ np.ones(nActions) == 1)

    m.setObjective(sum(sum(P[u,:, a] @ y * u_dist[u] for u in range(nU))*pi_e[s,a] for a in range(nActions)), GRB.MINIMIZE)

    m.params.NonConvex = 2
    m.params.OptimalityTol = 1e-8
    m.params.FeasibilityTol = 1e-8
    m.params.BarConvTol = 1e-8
    m.params.BarQCPConvTol = 1e-8
    m.params.PoolSearchMode = 2

    m.update()
    m.optimize()
    
    #print(m.IsQCP)
    
    #print(GRB.OPTIMAL)
    
    res_x = np.array([m.getVars()[i].x for i in range((nStates*nU+nU)*nActions)])
    return m.objVal, res_x

In [ ]:
def fixed_u_gp_s_rect(f, pi_e, u_param, Phat, pihat, P_bound, pi_bound, mdp):
    nStates = mdp.n_states
    nActions = mdp.n_actions
    nU = mdp.n_confound
    
    R_pi = np.zeros((nStates, nStates))
    for s in range(nStates):
        for sp in range(nStates):
            R_pi[s,sp] = pi_e[s] @ mdp.R[:,s,sp] 
            
    P_est = np.zeros((nU, nActions, nStates, nStates))
    pi_est = np.zeros((nU, nActions, nStates))
            
    Vworst = np.zeros(nStates)
    for x in range(nStates):
        y = np.array([R_pi[x,xp] + mdp.gamma * f[xp] for xp in range(nStates)])
        worst, vec = fixed_u_gp_s_rect_s(x, y, pi_e, u_param, Phat, pihat, P_bound, pi_bound, mdp)
        Vworst[x] = worst
        
        Pvec = vec[:nU*nStates*nActions].reshape((nU,nStates,nActions))
        P_est[:,:,x,:] = np.swapaxes(Pvec, 1, 2)
        
    return Vworst, P_est

In [ ]:
tightness_envs = []
mb_envs = []

sens = [(2,2), (10,10)]
nSens = len(sens)

hadd = [0, 20, 50, 200, 500]

for mdp , pi_b, pi_e, base_horizon, gamma, nStates, nActions, term in envs:
    
    for h in hadd:
        horizon = base_horizon + h
    
        print("---")

        print("running env with horizon " + str(horizon))

        dataset = confound_mdp.collect_sample(5000, mdp, pi_b, horizon)
        data = dataset.reshape((dataset.shape[0]*dataset.shape[1],5))

        Phat = confound_ope.estimate_P(dataset, mdp)
        pihat = confound_ope.estimate_pi(dataset, mdp)
        for a in range(nActions):
            for s in range(nStates):
                if Phat[a,s].sum() == 0:
                    Phat[a,s,term] = 1
                if pihat[s].sum() == 0:
                    pihat[s,:] = 1/nActions
        pi_avg = pi_b[0] * u_dist[0] + pi_b[1] * u_dist[1]

        print("now running robust mdps...")
        mb_results = np.zeros(nSens)
        tightness_results = np.zeros(nSens)

        for i,sensParams in tqdm(enumerate(sens)):
            gam, P_bound = sensParams

            V0 = np.zeros(nStates)
            fixed_u_v = V0.copy()
            for t in range(horizon):
                fixed_u_v, P_est = fixed_u_gp_s_rect(fixed_u_v, pi_e, 0.50, Phat, pihat, P_bound, gam, mdp)
            mb_results[i] = fixed_u_v @ mdp.x_dist

            # test tightness of proposed P
            test_model = confound_mdp.ConfoundMDP(P_est, mdp.R, mdp.x_dist, np.array([0.5,0.5]), mdp.gamma)
            Q0 = np.zeros((nStates, nActions))
            testQ = Q0.copy()
            for t in range(horizon):
                testQ = test_model.bellman_eval_update(testQ, np.array([pi_e,pi_e]))
            tightness_results[i] = test_model.get_value(testQ, pi_e)[1]

        mb_envs.append(mb_results)
        tightness_envs.append(tightness_results)

In [ ]:
(np.array(tightness_envs) - np.array(mb_envs)).T[0] #/ np.array(mb_envs)

In [ ]:
(np.array(tightness_envs) - np.array(mb_envs)).T[1] #/ np.array(mb_envs)